# Yan-Ferrighetto Project 2
### Project name: NYC City Bike Rideshare Study and Forecasting
### Project members: Xi Yan, Marissa Ferrighetto

Recording Link: https://cmu.box.com/s/tg6iev9omnd9144kxvhk0lwzduzx3dfl

# Motivation

As Heinz students, studying public policy and information systems, we are interested in how we can apply data science to public problems. We are interested in public transit and commuter patterns through cities. When we saw the expansive data set from New York City’s bike-share system, we were interested in applying machine learning techniques to the data.

# Domain & Overview

We plan on applying classification modeling for this project. The classification models can help drive funding decisions and allow stakeholders to better understand their user base.

As we discussed in the lecture, “A Computer Program is said to learn from experience with respect to some class of tasks T and performance measure P, if its performance at tasks in T, as measured by P, improves with E.” In the context of this machine learning project, the T tasks represents the classification modeling, E experience is the training and fitting of the models, and P performance are the results (such as accuracy) of the models.

For this project, we will measure performance based on the different tasks, T. For example, we will compare the accuracy and precision of the classifications.
 
Our goal is to find a feasible model that can be beneficial to potentially answering the questions that we proposed below. Our objective, for the performance, P, of the models is, to achieve over 50% of accuracy or other relevant metrics when doing a binary classification and over 100/n% when we are doing n numbers classification.

Concisely stated, our goal is to achieve P(T, E+ΔE) > P(T,E). 

T: classification modeling

E: training and model fitting

P: accuracy of the models


# Dataset

### Data Source
We downloaded dataset of [Citi Bike rideshare data](https://ride.citibikenyc.com/system-data) in the Citi Bike website.

### Dataset Description
- The data is stored monthly (total 37 CSV files)
- Each month is a separate CSV file
- Rows: around 50000 for each CSV file (Sum of about 2 million rows)
- Columns: 13
- The data itself is cleaned. It is also expandable to look at the distance between the start station and the end station

### Dataset Features
- Ride ID
- Rideable type (Classic Bike or Electric Bike)
- Started at (MM/DD/YY H:MM:SS)
- Ended at (MM/DD/YY H:MM:SS)
- Start station name
- Start station ID
- End station name
- End station ID
- Start latitude
- Start longitude
- End latitude
- End Longitude
- Member or casual ride (Causal or Member)


## Related Work

1. [Citi Bike Struggles to Keep Up With New Yorkers’ Love of Cycling](https://www.nytimes.com/2021/12/02/nyregion/citi-bike-parking-docking-station.html)

  In this article, the author describes the growth of the New York City bike share post-pandemic era and the pain spot of the system. There are demand problems that do not meet the increasing popularity of rideshare and also distribution problems although the Citi bike rideshare system has already deployed hundreds of thousands of bikes, there are situations where users are not able to find bikes at destined bike stops. This implies that a study about the usage of rides to solve the redistribution problem might be urged.

2. [Survey Says: City Voters Support Public Money Invested in Bike Share](https://nyc.streetsblog.org/2021/05/11/survey-says-city-voters-support-public-money-invested-in-bike-share/)

  This article talks about the significance and public support for increasing subsidies and public funding for the New York City rideshare. New York City is the most crowded city in the U.S., and if having as many cars per household as other cities, the traffic situation would become worse. As the article mentioned, “we need to learn more and more into — opening up public spaces, getting out of our cars, focusing more on public transportation. This is the way of the future, unquestionably” This also leaves a question for public transportation developers - how to spend the money most efficiently so that can best serve the public needs of New York City citizens?

3. [New York 25x25 Plan](https://nyc25x25.org/)

  This website is the main page of the advocacy, the New York 25x25 plan. This plan asks mayoral candidates to commit to dedicating 25 percent of the space now designated for vehicles — including 19,000 miles of roads and three million on-street parking spaces — as space for people by 2025 so that New Yorkers might have room to recover from (and thrive after) the COVID-19 pandemic, which has highlighted the city’s lack of equity when it comes to active transportation and green space. 


# What is the problem?

Increased demand for city bikes since the beginning of the COVID-19 pandemic lead to

**Distribution problem**: There are excess amount of bikes concentrated in some end stations where people generally find no bikes available in many stations. Redistribution is needed to address the supply and demand.

**Business problem**: How to direct increased funding as the bike program continues? Who uses the bike share problem? How can the program build user personas to segment and target users


# Questions that we want to achieve P(T, E+ΔE) > P(T,E).

1. How should the citibike act to respond to the first news, where there are distribution problems for bike share, to correctly redistribute the most needed types of bikes back to the station which lack bikes?

2. To meet the increasing demand for bike share and the New York 25x25 policy plan, how should citibike market to accurately identify, locate, and attract potential users to subscribe to the program?

# Part 1: Data Loading & Preparation

1.1 Load the data

Install the following packages if not already installed

In [ ]:
#!pip install geopy
#!pip install imblearn

Import required packages

In [ ]:
import glob
import pandas as pd
import numpy as np
import os

In [ ]:
from sklearn.preprocessing import LabelEncoder
from geopy import distance
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The NYC bikeshare data is from February of 2021 to March 2022. We originally planned on utilizing data beginning when the COVID-19 pandemic shutdowns began in the U.S. in March 2020, but unfortunately the data collected changed in January 2021, creating continuity issues. The dataset has 743,163 records and 13 columns.

Read CSV files

In [ ]:
joined_files = os.path.join("./drive/MyDrive/95885 Project 2/Source/", "JC*.csv")
joined_list = glob.glob(joined_files)
pre_df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
pre_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,121DD7DD23CB1335,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Hoboken Ave at Monmouth St,JC105,Christ Hospital,JC034,40.735208,-74.046964,40.734786,-74.050444,member
1,FD73FB85F008349D,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,Newport Pkwy,JC008,Marin Light Rail,JC013,40.728744,-74.032108,40.714584,-74.042817,member
2,39F9E6663CB5FDF6,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,Journal Square,JC103,Baldwin at Montgomery,JC020,40.733670,-74.062500,40.723659,-74.064194,member
3,A64745CB0792EC6F,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,casual
4,75CC76EB9543764A,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,member


In [ ]:
pre_df.shape

(743074, 13)

1.1 Preprocessing

In [ ]:
pre_df = pre_df.drop(['ride_id', 'start_station_name', 'end_station_name'], 1)
pre_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,rideable_type,started_at,ended_at,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,JC105,JC034,40.735208,-74.046964,40.734786,-74.050444,member
1,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,JC008,JC013,40.728744,-74.032108,40.714584,-74.042817,member
2,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,JC103,JC020,40.733670,-74.062500,40.723659,-74.064194,member
3,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,JC105,JC105,40.735208,-74.046963,40.735208,-74.046964,casual
4,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,JC105,JC105,40.735208,-74.046963,40.735208,-74.046964,member


Add feature for ZipCode and City

In the following steps, we have 3 csv file that contains ZipCode, City/Borough by latitude and latitude of New York City and its surburb areas

Read location CSV

In [ ]:
a = pd.read_csv('./drive/MyDrive/95885 Project 2/Source/out.csv')
a['ZipCode'] = a['address'].str[:5].astype(int)
a

,Unnamed: 0,lat,lng,address,ZipCode
0,1,40.721630,-74.049967,07302,7302
1,2,40.749984,-74.027150,07030,7030
2,3,40.745983,-74.028199,07030,7030
3,12,40.721630,-74.049968,07302,7302
4,28,40.745984,-74.028199,07030,7030
...,...,...,...,...,...
430,714140,40.752271,-73.987706,10018,10018
431,723276,40.801343,-73.971146,10025,10025
432,724586,40.861560,-73.912190,10468,10468
433,736040,40.705945,-74.013219,10006,10006


Read NY and NJ zip codes CSV

In [ ]:
ny = pd.read_csv('./drive/MyDrive/95885 Project 2/Source/nyc-zip-codes.csv')
ny['City'] = ny['Borough']
nj = pd.read_csv('./drive/MyDrive/95885 Project 2/Source/nj-zip-codes.csv')
zip_city = pd.concat([ny, nj])
zip_city = zip_city[['ZipCode', 'City']]
zip_city

,ZipCode,City
0,10453,Bronx
1,10457,Bronx
2,10460,Bronx
3,10458,Bronx
4,10467,Bronx
...,...,...
166,7036,Linden
167,7065,Rahway
168,7203,Roselle
169,7204,Roselle Park


In [ ]:
b = pd.merge(a, zip_city, how='left', on='ZipCode')
b = b[['lat', 'lng', 'ZipCode', 'City']]
b

,lat,lng,ZipCode,City
0,40.721630,-74.049967,7302,Jersey City
1,40.749984,-74.027150,7030,Hoboken
2,40.745983,-74.028199,7030,Hoboken
3,40.721630,-74.049968,7302,Jersey City
4,40.745984,-74.028199,7030,Hoboken
...,...,...,...,...
433,40.752271,-73.987706,10018,Manhattan
434,40.801343,-73.971146,10025,Manhattan
435,40.861560,-73.912190,10468,Bronx
436,40.705945,-74.013219,10006,Manhattan


The following process joins the city and zipcode CSV by start_lat/lng and end_lat/lng

In [ ]:
pre_df = pre_df.merge(b, how='left', left_on=['start_lat', 'start_lng'], right_on=['lat', 'lng'])

In [ ]:
pre_df['start_zip'] = pre_df['ZipCode']
pre_df['start_city'] = pre_df['City']
pre_df = pre_df.drop(['lat','lng','ZipCode','City'], axis=1)

In [ ]:
pre_df = pre_df.merge(b, how='left', left_on=['end_lat', 'end_lng'], right_on=['lat', 'lng'])

In [ ]:
pre_df['end_zip'] = pre_df['ZipCode']
pre_df['end_city'] = pre_df['City']
pre_df = pre_df.drop(['lat','lng','ZipCode','City'], axis=1)

In [ ]:
pre_df

,rideable_type,started_at,ended_at,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_zip,start_city,end_zip,end_city
0,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,JC105,JC034,40.735208,-74.046964,40.734786,-74.050444,member,7310,Jersey City,7306.0,Jersey City
1,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,JC008,JC013,40.728744,-74.032108,40.714584,-74.042817,member,7310,Jersey City,7302.0,Jersey City
2,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,JC103,JC020,40.733670,-74.062500,40.723659,-74.064194,member,7306,Jersey City,7306.0,Jersey City
3,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,JC105,JC105,40.735208,-74.046963,40.735208,-74.046964,casual,7310,Jersey City,7310.0,Jersey City
4,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,JC105,JC105,40.735208,-74.046963,40.735208,-74.046964,member,7310,Jersey City,7310.0,Jersey City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743076,electric_bike,2022-03-03 16:47:15,2022-03-03 16:51:28,HB505,HB305,40.751867,-74.030377,40.747907,-74.038412,member,7030,Hoboken,7030.0,Hoboken
743077,classic_bike,2022-03-08 18:02:08,2022-03-08 18:14:41,HB505,JC098,40.751867,-74.030377,40.724294,-74.035483,member,7030,Hoboken,7302.0,Jersey City
743078,classic_bike,2022-03-07 18:54:55,2022-03-07 19:07:06,HB505,JC098,40.751867,-74.030377,40.724294,-74.035483,member,7030,Hoboken,7302.0,Jersey City
743079,classic_bike,2022-03-19 02:58:42,2022-03-19 03:14:37,HB505,JC098,40.751867,-74.030377,40.724294,-74.035483,casual,7030,Hoboken,7302.0,Jersey City


Distance

The following step calculates the straight distance for the start location and end location using [great circle algorithm](https://www.geeksforgeeks.org/great-circle-distance-formula/#:~:text=Given%20that%20the%20radius%20of%20a%20sphere%20is%2010%20km,%2B%20sin%20a%20sin%20b%5D.).

In [ ]:
pre_df[pre_df['end_lat'].isnull()]

,rideable_type,started_at,ended_at,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_zip,start_city,end_zip,end_city
87,docked_bike,2021-02-07 15:44:49,2021-02-08 16:44:41,JC084,NaN,40.714358,-74.066610,NaN,NaN,member,7304,Jersey City,NaN,NaN
768,docked_bike,2021-02-27 18:24:58,2021-02-27 19:47:28,JC094,NaN,40.727551,-74.071060,NaN,NaN,member,7302,Jersey City,NaN,NaN
770,docked_bike,2021-02-19 20:08:02,2021-02-20 11:54:07,JC013,NaN,40.714584,-74.042817,NaN,NaN,casual,7302,Jersey City,NaN,NaN
771,docked_bike,2021-02-13 17:33:20,2021-02-13 22:34:42,JC075,NaN,40.725685,-74.048790,NaN,NaN,member,7302,Jersey City,NaN,NaN
773,docked_bike,2021-02-11 07:22:37,2021-02-11 07:59:04,JC072,NaN,40.712418,-74.038525,NaN,NaN,member,7302,Jersey City,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733397,classic_bike,2022-03-25 23:47:50,2022-03-27 00:47:30,JC038,NaN,40.712774,-74.036486,NaN,NaN,casual,7302,Jersey City,NaN,NaN
734620,electric_bike,2022-03-25 02:48:37,2022-03-26 03:48:32,HB201,NaN,40.750604,-74.024020,NaN,NaN,casual,7030,Hoboken,NaN,NaN
735383,electric_bike,2022-03-21 01:03:41,2022-03-22 02:03:35,HB607,NaN,40.740973,-74.028603,NaN,NaN,casual,7030,Hoboken,NaN,NaN
736876,classic_bike,2022-03-04 16:43:34,2022-03-05 17:43:29,JC098,NaN,40.724294,-74.035483,NaN,NaN,member,7302,Jersey City,NaN,NaN


In [ ]:
def cal_distance(from_lat, from_lng, to_lat, to_lng):
    go = (from_lat, from_lng)
    to = (to_lat, to_lng)
    try:
        return distance.great_circle(go, to)
    except ValueError:
        return np.nan

pre_df['distance'] = pre_df.apply(lambda row: cal_distance(row.start_lat,row.start_lng, row.end_lat, row.end_lng), axis=1)

From the code below, we can see that some ride history does not have information about end station, so we are not able to calculate the distance, we will ignore these missing values for now and imput them during machine learning pipeline process

In [ ]:
pre_df[pre_df['end_lat'].isnull()]

,rideable_type,started_at,ended_at,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_zip,start_city,end_zip,end_city,distance
87,docked_bike,2021-02-07 15:44:49,2021-02-08 16:44:41,JC084,NaN,40.714358,-74.066610,NaN,NaN,member,7304,Jersey City,NaN,NaN,NaN
768,docked_bike,2021-02-27 18:24:58,2021-02-27 19:47:28,JC094,NaN,40.727551,-74.071060,NaN,NaN,member,7302,Jersey City,NaN,NaN,NaN
770,docked_bike,2021-02-19 20:08:02,2021-02-20 11:54:07,JC013,NaN,40.714584,-74.042817,NaN,NaN,casual,7302,Jersey City,NaN,NaN,NaN
771,docked_bike,2021-02-13 17:33:20,2021-02-13 22:34:42,JC075,NaN,40.725685,-74.048790,NaN,NaN,member,7302,Jersey City,NaN,NaN,NaN
773,docked_bike,2021-02-11 07:22:37,2021-02-11 07:59:04,JC072,NaN,40.712418,-74.038525,NaN,NaN,member,7302,Jersey City,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733397,classic_bike,2022-03-25 23:47:50,2022-03-27 00:47:30,JC038,NaN,40.712774,-74.036486,NaN,NaN,casual,7302,Jersey City,NaN,NaN,NaN
734620,electric_bike,2022-03-25 02:48:37,2022-03-26 03:48:32,HB201,NaN,40.750604,-74.024020,NaN,NaN,casual,7030,Hoboken,NaN,NaN,NaN
735383,electric_bike,2022-03-21 01:03:41,2022-03-22 02:03:35,HB607,NaN,40.740973,-74.028603,NaN,NaN,casual,7030,Hoboken,NaN,NaN,NaN
736876,classic_bike,2022-03-04 16:43:34,2022-03-05 17:43:29,JC098,NaN,40.724294,-74.035483,NaN,NaN,member,7302,Jersey City,NaN,NaN,NaN


In [ ]:
pre_df['distance'] = pre_df['distance'].astype(str).str[:-3].astype('float64', errors = 'ignore')

There are some null data from the above calculation, this is because the starting location is the exact end location, so we impute 0 for these data

In [ ]:
pre_df['distance'] = pre_df['distance'].fillna(0)

In [ ]:
pre_df[pre_df['end_lat'].isnull()]

,rideable_type,started_at,ended_at,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_zip,start_city,end_zip,end_city,distance
87,docked_bike,2021-02-07 15:44:49,2021-02-08 16:44:41,JC084,NaN,40.714358,-74.066610,NaN,NaN,member,7304,Jersey City,NaN,NaN,
768,docked_bike,2021-02-27 18:24:58,2021-02-27 19:47:28,JC094,NaN,40.727551,-74.071060,NaN,NaN,member,7302,Jersey City,NaN,NaN,
770,docked_bike,2021-02-19 20:08:02,2021-02-20 11:54:07,JC013,NaN,40.714584,-74.042817,NaN,NaN,casual,7302,Jersey City,NaN,NaN,
771,docked_bike,2021-02-13 17:33:20,2021-02-13 22:34:42,JC075,NaN,40.725685,-74.048790,NaN,NaN,member,7302,Jersey City,NaN,NaN,
773,docked_bike,2021-02-11 07:22:37,2021-02-11 07:59:04,JC072,NaN,40.712418,-74.038525,NaN,NaN,member,7302,Jersey City,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733397,classic_bike,2022-03-25 23:47:50,2022-03-27 00:47:30,JC038,NaN,40.712774,-74.036486,NaN,NaN,casual,7302,Jersey City,NaN,NaN,
734620,electric_bike,2022-03-25 02:48:37,2022-03-26 03:48:32,HB201,NaN,40.750604,-74.024020,NaN,NaN,casual,7030,Hoboken,NaN,NaN,
735383,electric_bike,2022-03-21 01:03:41,2022-03-22 02:03:35,HB607,NaN,40.740973,-74.028603,NaN,NaN,casual,7030,Hoboken,NaN,NaN,
736876,classic_bike,2022-03-04 16:43:34,2022-03-05 17:43:29,JC098,NaN,40.724294,-74.035483,NaN,NaN,member,7302,Jersey City,NaN,NaN,


Drop weird ridetype

In [ ]:
pre_df['rideable_type'].value_counts()

classic_bike              582589
docked_bike               141306
electric_bike              19185
motivate_dockless_bike         1
Name: rideable_type, dtype: int64

In [ ]:
pre_df = pre_df[(pre_df.rideable_type != 'motivate_dockless_bike')]

Add feature - Intra city - This feature shows whether the user ride a bike from one borough to another

As we execute the following code, we see some SettingWithCopyWarnings. After further [investigation](https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas), we confirmed that our code is running as expected and the warnings can be disregarded.

In [ ]:
pre_df['intra'] = np.where(pre_df['start_city']!=pre_df['end_city'], 1, 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
pre_df

,rideable_type,started_at,ended_at,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_zip,start_city,end_zip,end_city,distance,intra
0,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,JC105,JC034,40.735208,-74.046964,40.734786,-74.050444,member,7310,Jersey City,7306.0,Jersey City,0.29694367026456386,0
1,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,JC008,JC013,40.728744,-74.032108,40.714584,-74.042817,member,7310,Jersey City,7302.0,Jersey City,1.814826093100416,0
2,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,JC103,JC020,40.733670,-74.062500,40.723659,-74.064194,member,7306,Jersey City,7306.0,Jersey City,1.1223030649377475,0
3,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,JC105,JC105,40.735208,-74.046963,40.735208,-74.046964,casual,7310,Jersey City,7310.0,Jersey City,7.613773379973303e-05,0
4,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,JC105,JC105,40.735208,-74.046963,40.735208,-74.046964,member,7310,Jersey City,7310.0,Jersey City,7.613773379973303e-05,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743076,electric_bike,2022-03-03 16:47:15,2022-03-03 16:51:28,HB505,HB305,40.751867,-74.030377,40.747907,-74.038412,member,7030,Hoboken,7030.0,Hoboken,0.8074615435560847,0
743077,classic_bike,2022-03-08 18:02:08,2022-03-08 18:14:41,HB505,JC098,40.751867,-74.030377,40.724294,-74.035483,member,7030,Hoboken,7302.0,Jersey City,3.0960533549248908,1
743078,classic_bike,2022-03-07 18:54:55,2022-03-07 19:07:06,HB505,JC098,40.751867,-74.030377,40.724294,-74.035483,member,7030,Hoboken,7302.0,Jersey City,3.0960533549248908,1
743079,classic_bike,2022-03-19 02:58:42,2022-03-19 03:14:37,HB505,JC098,40.751867,-74.030377,40.724294,-74.035483,casual,7030,Hoboken,7302.0,Jersey City,3.0960533549248908,1


Start at end at

In [ ]:
pre_df[["started_at", "ended_at"]] = pre_df[["started_at", "ended_at"]].apply(pd.to_datetime)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
pre_df['duration'] = pre_df['ended_at']-pre_df['started_at']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
pre_df['duration_secs'] = pre_df['duration'].dt.total_seconds()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#Time of day and season are calculated using started_at derived values
[month%12 // 3 + 1 for month in range(1, 13)]
pre_df['season_num'] = pre_df['started_at'].dt.month%12 // 3 + 1
pre_df['season'] = pre_df['season_num'].map({1:'Winter',2:'Spring', 3: 'Summer', 4: 'Fall'})
pre_df['season'] = pre_df['season'].astype("string")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [ ]:
#Time of day and season are calculated using started_at derived values
pre_df['time_of_day'] = (pre_df['started_at'].dt.hour % 24 + 4) // 4
pre_df['time_of_day'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
pre_df

,rideable_type,started_at,ended_at,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,...,start_city,end_zip,end_city,distance,intra,duration,duration_secs,season_num,season,time_of_day
0,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,JC105,JC034,40.735208,-74.046964,40.734786,-74.050444,member,...,Jersey City,7306.0,Jersey City,0.29694367026456386,0,0 days 00:07:00,420.0,1,Winter,Night
1,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,JC008,JC013,40.728744,-74.032108,40.714584,-74.042817,member,...,Jersey City,7302.0,Jersey City,1.814826093100416,0,0 days 00:22:35,1355.0,1,Winter,Evening
2,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,JC103,JC020,40.733670,-74.062500,40.723659,-74.064194,member,...,Jersey City,7306.0,Jersey City,1.1223030649377475,0,0 days 00:06:21,381.0,1,Winter,Night
3,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,JC105,JC105,40.735208,-74.046963,40.735208,-74.046964,casual,...,Jersey City,7310.0,Jersey City,7.613773379973303e-05,0,0 days 00:24:19,1459.0,1,Winter,Evening
4,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,JC105,JC105,40.735208,-74.046963,40.735208,-74.046964,member,...,Jersey City,7310.0,Jersey City,7.613773379973303e-05,0,0 days 00:00:30,30.0,1,Winter,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743076,electric_bike,2022-03-03 16:47:15,2022-03-03 16:51:28,HB505,HB305,40.751867,-74.030377,40.747907,-74.038412,member,...,Hoboken,7030.0,Hoboken,0.8074615435560847,0,0 days 00:04:13,253.0,2,Spring,Evening
743077,classic_bike,2022-03-08 18:02:08,2022-03-08 18:14:41,HB505,JC098,40.751867,-74.030377,40.724294,-74.035483,member,...,Hoboken,7302.0,Jersey City,3.0960533549248908,1,0 days 00:12:33,753.0,2,Spring,Evening
743078,classic_bike,2022-03-07 18:54:55,2022-03-07 19:07:06,HB505,JC098,40.751867,-74.030377,40.724294,-74.035483,member,...,Hoboken,7302.0,Jersey City,3.0960533549248908,1,0 days 00:12:11,731.0,2,Spring,Evening
743079,classic_bike,2022-03-19 02:58:42,2022-03-19 03:14:37,HB505,JC098,40.751867,-74.030377,40.724294,-74.035483,casual,...,Hoboken,7302.0,Jersey City,3.0960533549248908,1,0 days 00:15:55,955.0,2,Spring,Late Night


Now, the preprocess is finished, and this dataset is ready for label encoding (turn string into categorical values)

Here we drop many unuseful features, these are string variables that will drag down the performance of the features, and we don't want to have too many features for OneHotEncoding, which significantly increase model training time.

In [ ]:
pre_df = pre_df.drop(['duration', 'started_at', 'ended_at', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'start_zip', 'end_zip', 'end_station_id', 'end_city', 'start_city', 'season_num'], axis=1)

We have to replace the distance we ignore above and imput with NaN value now, and then we will handle them but imputing in the pipeline next

In [ ]:
pre_df['distance'] = pre_df['distance'].replace('', np.NAN)
pre_df[pre_df['distance'].isnull()]

,rideable_type,start_station_id,member_casual,distance,intra,duration_secs,season,time_of_day
87,docked_bike,JC084,member,NaN,1,89992.0,Winter,Noon
768,docked_bike,JC094,member,NaN,1,4950.0,Winter,Evening
770,docked_bike,JC013,casual,NaN,1,56765.0,Winter,Night
771,docked_bike,JC075,member,NaN,1,18082.0,Winter,Evening
773,docked_bike,JC072,member,NaN,1,2187.0,Winter,Early Morning
...,...,...,...,...,...,...,...,...
733397,classic_bike,JC038,casual,NaN,1,89980.0,Spring,Night
734620,electric_bike,HB201,casual,NaN,1,89995.0,Spring,Late Night
735383,electric_bike,HB607,casual,NaN,1,89994.0,Spring,Late Night
736876,classic_bike,JC098,member,NaN,1,89995.0,Spring,Evening


Final step - label encode all of the non numeric features

In [ ]:
non_numeric_cols = ['member_casual', 'time_of_day', 'rideable_type', 'start_station_id', 'season']
for col in non_numeric_cols:
    pre_df[col] = LabelEncoder().fit_transform(pre_df[col].values)

pre_df

,rideable_type,start_station_id,member_casual,distance,intra,duration_secs,season,time_of_day
0,1,80,1,0.29694367026456386,0,420.0,3,4
1,1,35,1,1.814826093100416,0,1355.0,3,1
2,1,78,1,1.1223030649377475,0,381.0,3,4
3,1,80,0,7.613773379973303e-05,0,1459.0,3,1
4,1,80,1,7.613773379973303e-05,0,30.0,3,4
...,...,...,...,...,...,...,...,...
743076,2,24,1,0.8074615435560847,0,253.0,1,1
743077,0,24,1,3.0960533549248908,1,753.0,1,1
743078,0,24,1,3.0960533549248908,1,731.0,1,1
743079,0,24,0,3.0960533549248908,1,955.0,1,2


Now all the data are prepared and ready for machine learning!

# Part 2: Classification Question 1

Based on the time spent, start station, end station, and distance, can we classify bike type (dock, electric, manual)? 

Split train test dataset

In [ ]:
train_set_p2, test_set_p2 = train_test_split(pre_df, test_size=0.2, random_state=35)

## In train dataset:

Drop Unuseful features

Drop duplicates

In [ ]:
dups = train_set_p2.duplicated()
train_set_p2[dups]

,rideable_type,start_station_id,member_casual,distance,intra,duration_secs,season,time_of_day
315372,0,14,0,0.82158948243469,0,264.0,2,1
633760,0,33,1,0.5567827596309259,0,202.0,3,1
317713,0,33,1,0.0,0,17.0,2,5
529483,0,55,1,0.6924918349344047,0,203.0,0,5
411012,0,62,0,0.49547366538293386,0,146.0,0,1
...,...,...,...,...,...,...,...,...
657307,0,33,1,0.66701962691726,0,237.0,3,1
469711,0,33,1,0.9412787868416884,0,264.0,0,5
573052,0,81,1,0.2907568150588054,0,135.0,0,5
262056,0,29,1,0.8407219513223058,0,275.0,2,1


In [ ]:
train_set_p2.drop_duplicates(inplace=True)

Check if there are any null values (We should only have the NaN distance values we mentioned before)

In [ ]:
train_set_p2.isnull().sum()

rideable_type          0
start_station_id       0
member_casual          0
distance            1913
intra                  0
duration_secs          0
season                 0
time_of_day            0
dtype: int64

Split X_train and y_train

In [ ]:
X_train_p2 = train_set_p2.drop('rideable_type', axis=1)
y_train_p2 = train_set_p2['rideable_type'].copy()

In [ ]:
y_train_p2.value_counts()

0    446436
1    110895
2     15030
Name: rideable_type, dtype: int64

Y_train is heavily imbalanced

Class Imbalance (We use undersampler to downsample since we have so many data, if we upsample, the model takes forever to run)

In [ ]:
rus_p2 = RandomUnderSampler(random_state=35)

X_train_p2, y_train_p2 = rus_p2.fit_resample(X_train_p2, y_train_p2)

### Pipeline - num and cat preprocessor

Use MICE imputer to impute the missing distance value

Use standard scaler to scale the numeric value

In [ ]:
num_pipeline_p2 = Pipeline([
        ('imp', IterativeImputer()),
        ('std_scaler', StandardScaler()),
    ])

Use OneHotEncoder to handle all of the categorical features

In [ ]:
num_attribs = ['distance', 'duration_secs']
cat_attribs = ['member_casual', 'time_of_day', 'season', 'intra', 'start_station_id']

preprocessor_p2 = ColumnTransformer([
        ('num', num_pipeline_p2, num_attribs),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_attribs),
    ])

## Train models

Random Forest

The reason to use RF in this part, also in part 3 is that Random Forest provide us a very flexible model tuning and customizable parameters that we could alter to find the best model for this scenario.

In [ ]:
rf_p2 = Pipeline(
    steps=[
        ('preprocessor', preprocessor_p2),
        ('classifier', RandomForestClassifier(random_state=35))
    ]
)

param_grid_rf_p2 = [
    {'classifier__n_estimators': [50, 100]},
  ]

In [ ]:
grid_search_rf_p2 = GridSearchCV(rf_p2, param_grid_rf_p2, cv=4)
grid_search_rf_p2.fit(X_train_p2, y_train_p2)

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imp',
                                                                                          IterativeImputer()),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['distance',
                                                                          'duration_secs']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                  

KNN

The reason to use KNN in this part of question is that we want to understand if the user behavior when riding different types of bike are exhibitted in clusters so that we might have have a better performance than using RF.

In [ ]:
knn_p2 = Pipeline(
    steps=[
        ('preprocessor', preprocessor_p2),
        ('classifier', KNeighborsClassifier())
    ]
)

param_knn_p2 = [
    {'classifier__n_neighbors': [4, 8, 12]},
]

In [ ]:
grid_search_knn_p2 = GridSearchCV(knn_p2, param_knn_p2, cv=3)
grid_search_knn_p2.fit(X_train_p2, y_train_p2)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imp',
                                                                                          IterativeImputer()),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['distance',
                                                                          'duration_secs']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                  

## Test set - doing exactly as the train dataset (without downsample y_test)

Drop duplicated rows

In [ ]:
dups = test_set_p2.duplicated()
dups.any()

True

In [ ]:
test_set_p2.drop_duplicates(inplace=True)

Check if there are null values, (impute during pipeline just like train sets)

In [ ]:
test_set_p2.isnull().sum()

rideable_type         0
start_station_id      0
member_casual         0
distance            516
intra                 0
duration_secs         0
season                0
time_of_day           0
dtype: int64

Split X_test and y_test

In [ ]:
X_test_p2 = test_set_p2.drop('rideable_type', axis=1)
y_test_p2 = test_set_p2['rideable_type'].copy()

## Model performance

RandomForest

In [ ]:
final_model_rf_p2 = grid_search_rf_p2.best_estimator_

print(classification_report(y_test_p2, final_model_rf_p2.predict(X_test_p2)))

              precision    recall  f1-score   support

           0       0.95      0.81      0.87    114998
           1       0.50      0.67      0.57     28187
           2       0.28      0.85      0.42      3885

    accuracy                           0.78    147070
   macro avg       0.58      0.77      0.62    147070
weighted avg       0.85      0.78      0.80    147070



KNN

In [ ]:
final_model_knn_p2 = grid_search_knn_p2.best_estimator_

print(classification_report(y_test_p2, final_model_knn_p2.predict(X_test_p2)))

              precision    recall  f1-score   support

           0       0.93      0.80      0.86    114998
           1       0.43      0.52      0.47     28187
           2       0.20      0.77      0.32      3885

    accuracy                           0.74    147070
   macro avg       0.52      0.70      0.55    147070
weighted avg       0.82      0.74      0.77    147070



As we can see above, our trained model outperformed the baselines (0.33) by over 100%. The performance of Random Forest is better than K neared neighbors algorithm. 

This model could be used to help the citi bike program classify which bike types to distribute to a specific start station when they are facing a distribution problem. Additionally, this model can be used to guide the types of bikes to invest in when there is extra capital.

How to add value to Citibike: This model has given the Citibike, with high accuracy, to classify which types of bikes (classic, docked, and ebike) should be redistributed to which start location of the city.

Therefore, to address the first question of our project, we are able to use this model during the redistribution process. For example, if the city collects different types of bikes from the end stations, which have concentrated excessive bikes, then, they can redistribute different types of bikes back to different start location based on the season, time of the day. Also, if a truck driver for Citibike is driving a truck with loaded different types of bikes, your chances of dropping those bikes to the correct location is expected to be 100% more correct than randomly guessing.

# Part 3: Classification Question 2

Based on predictive analysis, can we predict the member type (subscriber vs. non-subscriber?

Split train and test sets for part 3

In [ ]:
train_set_p3, test_set_p3 = train_test_split(pre_df, test_size=0.3, random_state=35)

## In Train Set

In [ ]:
train_set_p3['member_casual'].value_counts()

1    296306
0    223850
Name: member_casual, dtype: int64

The size of this train set is too large. If we train on this amount of data, it will makes the model run for many hours. So we sample the data first to lower the size. 

Notice that sampling might decrease the fitness of the model, but this is something we have to compromise since the cost of waiting in this project is high. If we have more resources or computational power, we could perform with all of the train set data and reevaluate the analysis.

In [ ]:
train_set_p3_sampled = train_set_p3.sample(frac=0.2, replace=True, random_state=35)

In [ ]:
train_set_p3_sampled['member_casual'].value_counts()

1    59537
0    44494
Name: member_casual, dtype: int64

Now, the train set is within a reasonable size, we are able to perform training based on this sample.

In [ ]:
X_train_p3 = train_set_p3_sampled.drop('member_casual', axis=1)
y_train_p3 = train_set_p3_sampled['member_casual'].copy()

Next, we address the class imbalance.

In [ ]:
rus_p3 = RandomUnderSampler(random_state=35)

X_train_p3, y_train_p3 = rus_p3.fit_resample(X_train_p3, y_train_p3)

In [ ]:
y_train_p3.value_counts()

0    44494
1    44494
Name: member_casual, dtype: int64

### Pipeline - num and cat preprocessor

We are using the MICE imputer to impute the missing distance value.

We are using the standard scaler to scale the numeric value.

In [ ]:
num_pipeline_p3 = Pipeline([
        ('imp', IterativeImputer()),
        ('std_scaler', StandardScaler()),
    ])

Next, we use OneHotEncoder to address all of the categorical features.

In [ ]:
num_attribs = ['distance', 'duration_secs']
cat_attribs = ['rideable_type', 'season', 'intra', 'start_station_id', 'time_of_day']

preprocessor_p3 = ColumnTransformer([
        ('num', num_pipeline_p3, num_attribs),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_attribs),
    ])

First, we try a random forest model.

In [ ]:
rf_p3 = Pipeline(
    steps=[
        ('preprocessor', preprocessor_p3),
        ('classifier', RandomForestClassifier(random_state=35))
    ]
)

param_grid_rf_p3 = [
    {'classifier__n_estimators': [50, 100]},
  ]

In [ ]:
grid_search_rf_p3 = GridSearchCV(rf_p3, param_grid_rf_p3, cv=4)

grid_search_rf_p3.fit(X_train_p3, y_train_p3)

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imp',
                                                                                          IterativeImputer()),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['distance',
                                                                          'duration_secs']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                  

Next, we try logistic regression.

The reason why logistic regression might be a good model for this problem is that the membership type is a binary, and logistic regreesion is a relatively fast model for this very large amount of dataset.

In [ ]:
lr_p3 = Pipeline(
    steps=[
        ('preprocessor', preprocessor_p3),
        ('classification', LogisticRegression())
    ]
)

param_range_lr_fl = [1.0, 0.5]

param_lr_p3 = [
    {'classification__penalty': ['l1', 'l2'],
     'classification__C': param_range_lr_fl,
     'classification__solver': ['liblinear']}
]

In [ ]:
grid_search_lr_p3 = GridSearchCV(lr_p3, param_lr_p3, cv=4)
grid_search_lr_p3.fit(X_train_p3, y_train_p3)

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imp',
                                                                                          IterativeImputer()),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['distance',
                                                                          'duration_secs']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                  

## Test set - doing exactly as the train dataset (without downsample y_test)

Next, we drop duplicated rows.

In [ ]:
dups = test_set_p3.duplicated()
dups.any()

True

In [ ]:
test_set_p3.drop_duplicates(inplace=True)

Check if there are null values, and impute during pipeline just like in the training sets.

In [ ]:
test_set_p3.isnull().sum()

rideable_type         0
start_station_id      0
member_casual         0
distance            761
intra                 0
duration_secs         0
season                0
time_of_day           0
dtype: int64

Next, we split X_test and y_test.

In [ ]:
X_test_p3 = test_set_p3.drop('member_casual', axis=1)
y_test_p3 = test_set_p3['member_casual'].copy()

## Model Performance

RandomForest

In [ ]:
final_model_rf_p3 = grid_search_rf_p3.best_estimator_

print(classification_report(y_test_p3, final_model_rf_p3.predict(X_test_p3)))

              precision    recall  f1-score   support

           0       0.59      0.66      0.62     95499
           1       0.71      0.64      0.67    124050

    accuracy                           0.65    219549
   macro avg       0.65      0.65      0.65    219549
weighted avg       0.66      0.65      0.65    219549



LogisticRegression

In [ ]:
final_model_lr_p3 = grid_search_lr_p3.best_estimator_

print(classification_report(y_test_p3, final_model_lr_p3.predict(X_test_p3)))

              precision    recall  f1-score   support

           0       0.58      0.63      0.60     95499
           1       0.70      0.64      0.67    124050

    accuracy                           0.64    219549
   macro avg       0.64      0.64      0.64    219549
weighted avg       0.64      0.64      0.64    219549



As we can see above, our trained model outperformed the baselines. The performance of Random Forest is slightly better than the logistic regression.

Our second question asked: How should citibike market to attract potential users so that it meets the increasing demand for bike share and the New York 25x25 policy plan?

This model can help the citibike stakeholders to begin to build user personas by understanding what types users use the bikes. The user experience team can conduct user interviews with a subset of riders who use the service for different purposes. Once the personas are created, they can be used to build target audiences for advertising and marketing. They can also be used to validate future application or bike docking station design decisions.

## Conclusion

As sustainability and alternate modes of transportation increase in importance, the scalability and investment decisions of bikeshare programs in large metropolitan areas such as NYC are incredibly important. Data science, specifically machine learning, is a great tool to drive decision making and understand users in very large data sets.

Our models could be expanded upon and utilized to improve processes and drive growth of the program. The models for the first classification question can help classify which bike types to distribute to a specific start station when they are facing a distribution problem. Additionally, this model can be used to guide the types of bikes to invest in when there are new funding sources.

Based off of the results of the models for the second research question, citibike stakeholders can create user personas and targeted advertising campaigns to grow ridership. This will facilitate a more efficient use of advertising dollars for the expansion.



## Future Work

Given more time, we can train models with GridSearchCV by tuning more parameters to improve the performance.

Given more resources, such as computational power, we can upsample the class imbalance instead of downsampling and we can skip the sampling process to reduce size in part 3.

We can also use other resample techniques instead of using pure random resample methods to improve performance.

We can continuously to test and train using the newest data from Citibike (publishes monthly) to update our models.
